In [10]:
import unittest
import yaml
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

class TestCharStats(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        # Load configuration from config.yaml
        with open("config.yaml", "r") as config_file:
            config = yaml.safe_load(config_file)

        # Create a Spark session
        cls.spark = SparkSession.builder.appName("TestCharStats").getOrCreate()
        
        # Assign configuration variables
        cls.table_name = config["table_name"]
        cls.primary_key_columns = config["primary_key_columns"]
        cls.threshold_percentage = config["threshold_percentage"]

    @classmethod
    def tearDownClass(cls):
        # Stop the Spark session
        cls.spark.stop()

    def test_primary_key(self):
        # Load your DataFrame
        df = self.spark.read.parquet("/home/jovyan/work/digital/target/char_stats/")  # Replace with the actual path

        # Check if the primary key columns have no duplicates
        num_rows = df.count()
        num_distinct_rows = df.select(*self.primary_key_columns).distinct().count()
        self.assertEqual(num_rows, num_distinct_rows)

    def test_count_increase(self):
        # Load your DataFrames for previous and current loads
        df_previous = self.spark.read.parquet("/home/jovyan/work/digital/target/char_stats/")
        df_current = self.spark.read.parquet("/home/jovyan/work/digital/target/char_stats/")  # Replace with the actual path

        # Calculate the count increase percentage
        count_previous = df_previous.count()
        count_current = df_current.count()
        increase_percentage = (count_current - count_previous) / count_previous * 100

        # Check if the count increase is not less than the threshold percentage
        self.assertLessEqual(increase_percentage, self.threshold_percentage)

# Load and run the tests from Jupyter Notebook
test_suite = unittest.TestLoader().loadTestsFromTestCase(TestCharStats)
unittest.TextTestRunner(verbosity=2).run(test_suite)


test_count_increase (__main__.TestCharStats.test_count_increase) ... ok
test_primary_key (__main__.TestCharStats.test_primary_key) ... FAIL

FAIL: test_primary_key (__main__.TestCharStats.test_primary_key)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_20551/1710263554.py", line 33, in test_primary_key
    self.assertEqual(num_rows, num_distinct_rows)
AssertionError: 198 != 197

----------------------------------------------------------------------
Ran 2 tests in 2.641s

FAILED (failures=1)


<unittest.runner.TextTestResult run=2 errors=0 failures=1>

In [13]:
import unittest
import yaml
from pyspark.sql import SparkSession
import pandas as pd
from datetime import datetime

# Create a list to store test results
test_results = []

class TestCharStats(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        # Load configuration from config.yaml
        with open("config.yaml", "r") as config_file:
            config = yaml.safe_load(config_file)

        # Create a Spark session
        cls.spark = SparkSession.builder.appName("TestCharStats").getOrCreate()
        
        # Assign configuration variables
        cls.table_name = config["table_name"]
        cls.primary_key_columns = config["primary_key_columns"]
        cls.threshold_percentage = config["threshold_percentage"]

    @classmethod
    def tearDownClass(cls):
        # Stop the Spark session
        cls.spark.stop()

    def test_primary_key(self):
        # Load your DataFrame
        df = self.spark.read.parquet("/home/jovyan/work/digital/target/char_stats/")  # Replace with the actual path

        # Check if the primary key columns have no duplicates
        num_rows = df.count()
        num_distinct_rows = df.select(*self.primary_key_columns).distinct().count()
        self.assertEqual(num_rows, num_distinct_rows)

    def test_count_increase(self):
        # Load your DataFrames for previous and current loads
        df_previous = self.spark.read.parquet("/home/jovyan/work/digital/target/char_stats/")
        df_current = self.spark.read.parquet("/home/jovyan/work/digital/target/char_stats/")  # Replace with the actual path

        # Calculate the count increase percentage
        count_previous = df_previous.count()
        count_current = df_current.count()
        increase_percentage = (count_current - count_previous) / count_previous * 100

        # Check if the count increase is not less than the threshold percentage
        self.assertLessEqual(increase_percentage, self.threshold_percentage)

if __name__ == '__main__':
    # Create a Spark session
    spark = SparkSession.builder.appName("TestCharStats").getOrCreate()

    # Load configuration from config.yaml
    with open("config.yaml", "r") as config_file:
        config = yaml.safe_load(config_file)

    # Assign configuration variables
    table_name = config["table_name"]
    primary_key_columns = config["primary_key_columns"]
    threshold_percentage = config["threshold_percentage"]

    # Create a test suite
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestCharStats)

    # Run the tests
    test_runner = unittest.TextTestRunner()
    test_result = test_runner.run(test_suite)

    # Log test results
    for test_case, error in test_result.failures:
        test_results.append({
            "Test Case": test_case.id(),
            "Status": "Fail",
            "Timestamp": datetime.now()
        })

    for test_case, error in test_result.errors:
        test_results.append({
            "Test Case": test_case.id(),
            "Status": "Error",
            "Timestamp": datetime.now()
        })

    # Collect successful test cases
    all_test_cases = test_suite.countTestCases()
    failed_test_cases = len(test_result.failures) + len(test_result.errors)
    successful_test_cases = all_test_cases - failed_test_cases

    for i in range(successful_test_cases):
        test_results.append({
            "Test Case": f"Test {i+1}",
            "Status": "Pass",
            "Timestamp": datetime.now()
        })

    # Convert the test results to a DataFrame
    test_results_df = pd.DataFrame(test_results)

    # Write the DataFrame to a CSV file
    test_results_df.to_csv("test_results.csv", index=False)


.F
FAIL: test_primary_key (__main__.TestCharStats.test_primary_key)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_20551/1729191441.py", line 37, in test_primary_key
    self.assertEqual(num_rows, num_distinct_rows)
AssertionError: 198 != 197

----------------------------------------------------------------------
Ran 2 tests in 2.982s

FAILED (failures=1)
